In [ ]:
import spacy

In [ ]:
nlp = spacy.load('vfbr_persons_model')

In [ ]:
for x in VfbEntry.objects.all():
    doc = nlp(x.vollregest)
    try:
        if len(doc.ents) > 0:
            if len(doc.ents[0].text) > 5:
                pers, _ = Person.objects.get_or_create(legacy_id=doc.ents[0].text[:299])
            else:
                pers = None
        else:
            pers = None
    except Exception as e:
        print(e)
        pers = None
    if pers is not None:
        x.mentioned_person.add(pers)

In [ ]:
for x in Person.objects.all():
    x.written_name = x.legacy_id
    x.written_name_leven = x.legacy_id[:254]
    if len(x.legacy_id.split(' ')) > 1:
        x.forename = x.legacy_id.split(' ')[0]
        x.name = x.legacy_id.split(' ')[1]
    x.save()


In [ ]:
nlp = spacy.load('vfbr_places_model')

In [ ]:
for x in Person.objects.all():
    doc = nlp(x.written_name)
    if len(doc.ents) > 0:
        for ent in doc.ents:
            pl, _ = Place.objects.get_or_create(name=ent.text)
            x.belongs_to_place.add(pl)